In [23]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics.cluster import contingency_matrix 

In [24]:
%store -r test

# Take the name of the different classes (e.g. '0', '1', '2', ...)
classes = test.groupby(['label'])['label'].unique().index.to_list()

In [25]:
predicted_test = test.copy()

feature_1= 'x'
feature_2='y'
feature_3='arctan_1'

models = []
for n_class in classes: # Can be thrown an expection beacuse this is the labels of the test set and not of the models (train set)
    model = pickle.load(open('models_7/'+str(n_class)+'hmm_model.pkl', 'rb'))
    models.append(model)

#log_likelihoods = pd.DataFrame()
#log_likelihoods['predicted_class'] = classes
#log_like_max = []
#top_3_loglikelihood = []
top_3_predicted_classes = []
predicted_class = []
#is_predicted = []
true_label=[]

for i in range(len(test)):
    angle_1 = test[feature_1].iloc[i]
    angle_2 = test[feature_2].iloc[i]
    angle_3 = test[feature_3].iloc[i]
 
    X=np.array([angle_1,angle_2,angle_3]).T
    print(X)
    true_label.append(test['label'].iloc[i])
    list_log_likelihoods = []
    for model in models:
        
        list_log_likelihoods.append(model.score(X,lengths=len(X)))
        pass

    predicted_class.append(np.argmax(list_log_likelihoods))

    top_3_values = sorted(list_log_likelihoods)[-3:]
    top_3_predicted_classes.append([i for i, x in enumerate(list_log_likelihoods) if x == top_3_values[0] or x == top_3_values[1] or x == top_3_values[2]])
    

[[4.09e+02 1.36e+02 3.20e-01]
 [4.19e+02 1.48e+02 3.40e-01]
 [4.22e+02 1.59e+02 3.60e-01]
 [4.23e+02 1.71e+02 3.80e-01]
 [4.22e+02 1.87e+02 4.20e-01]
 [4.18e+02 2.05e+02 4.60e-01]
 [4.09e+02 2.37e+02 5.30e-01]
 [4.05e+02 2.49e+02 5.50e-01]
 [3.94e+02 2.82e+02 6.20e-01]
 [3.91e+02 2.93e+02 6.40e-01]
 [3.88e+02 3.19e+02 6.90e-01]
 [3.94e+02 3.33e+02 7.00e-01]
 [4.05e+02 3.45e+02 7.10e-01]
 [4.26e+02 3.52e+02 6.90e-01]
 [4.60e+02 3.57e+02 6.60e-01]
 [5.34e+02 3.58e+02 5.90e-01]
 [5.65e+02 3.58e+02 5.60e-01]
 [6.62e+02 3.55e+02 4.90e-01]
 [7.28e+02 3.55e+02 4.50e-01]
 [8.25e+02 3.55e+02 4.10e-01]
 [8.79e+02 3.55e+02 3.80e-01]
 [9.26e+02 3.55e+02 3.70e-01]
 [9.44e+02 3.54e+02 3.60e-01]
 [6.83e+02 1.56e+02 2.20e-01]
 [6.83e+02 1.58e+02 2.30e-01]
 [6.93e+02 1.77e+02 2.50e-01]
 [6.96e+02 1.87e+02 2.60e-01]
 [7.00e+02 2.22e+02 3.10e-01]
 [7.01e+02 2.36e+02 3.20e-01]
 [7.01e+02 2.86e+02 3.90e-01]
 [6.98e+02 3.23e+02 4.30e-01]
 [6.93e+02 3.61e+02 4.80e-01]
 [6.85e+02 4.13e+02 5.40e-01]
 [6.83e+02

In [26]:
print(predicted_class)
print(true_label)

print(len(predicted_class))
print(len(true_label))

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
[4, 1, 9, 3, 5, 8, 0, 4, 6, 0, 8, 4, 7, 2, 6, 4, 7, 3, 1, 2, 8, 0, 9, 0, 5, 3, 6, 4, 5, 2, 4, 5, 2, 0, 0, 4, 2, 1, 2, 9, 3, 2, 9, 3, 1, 7, 5, 5, 8, 6, 7, 4, 5, 3, 3, 1, 0, 9, 6, 8, 9, 0, 6, 9, 9, 8, 1, 0, 2, 4, 5, 2, 6, 9, 3, 4, 4, 2, 6, 0, 9, 9, 8, 7, 2,

In [27]:
matrix=contingency_matrix(true_label, predicted_class)
print(matrix)

[[ 0 34]
 [ 0 23]
 [ 0 23]
 [ 0 22]
 [ 0 25]
 [ 0 25]
 [ 0 25]
 [ 0 21]
 [ 0 24]
 [ 1 25]]


In [28]:

accuracy=np.sum(np.diag(matrix))/np.sum(matrix)
print("Accuracy: ", accuracy)

Accuracy:  0.09274193548387097


In [29]:
# Calculate top 3 accuracy
sum_top_3 = 0
for i in range(len(top_3_predicted_classes)):
    if true_label[i] in top_3_predicted_classes[i]:
        sum_top_3 += 1

accuracy_top_3 = sum_top_3/len(predicted_test["label"])
print("Accuracy top 3: ", accuracy_top_3)

Accuracy top 3:  0.2862903225806452
